In [1]:
import numpy as np

In [2]:
from sklearn.tree import DecisionTreeClassifier
from interpret.glassbox import ExplainableBoostingClassifier
from interpret.glassbox import DecisionListClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB


clfs = {
    "CART": DecisionTreeClassifier(random_state=1234),
    "EBM": ExplainableBoostingClassifier(),
    "LR_l2": LogisticRegression(penalty="l2",random_state=1234),
    "GNB": GaussianNB(),
    "LR": LogisticRegression(penalty="none", random_state=1234),
    "DL": DecisionListClassifier(random_state=1234)
    
}

In [3]:
results = np.load('test_results/auc/auc_results.npy')
print("\nScores:\n", results.shape)


Scores:
 (6, 12, 10)


In [4]:
mean_results = np.mean(results, axis=2).T
print("\nMean results:\n", mean_results)


Mean results:
 [[0.91590909 0.96071068 0.97088023 0.9257684  0.95712843 0.92681457]
 [0.79       0.83428571 0.82833333 0.74809524 0.84047619 0.53690476]
 [0.69006276 0.7251629  0.58751781 0.6596794  0.58788254 0.67504435]
 [0.68771429 0.6        0.81697959 0.71297959 0.82441837 0.5       ]
 [0.55984991 0.54702253 0.54285565 0.58393634 0.54285565 0.5825487 ]
 [0.67851567 0.70537037 0.73139316 0.72405983 0.72939316 0.72859544]
 [0.61985507 0.66160455 0.67374741 0.67024845 0.67516563 0.5       ]
 [0.72135714 0.7732381  0.78621429 0.78084524 0.7885119  0.719     ]
 [0.55456369 0.50206135 0.50227457 0.6607268  0.50227457 0.5006786 ]
 [0.5637361  0.64820474 0.68368772 0.66743485 0.67732153 0.50555556]
 [0.57079358 0.60341323 0.50053125 0.55200018 0.50053125 0.57345865]
 [0.80469075 0.88084428 0.87087986 0.69917077 0.86689381 0.87017515]]


In [5]:
from scipy.stats import rankdata
ranks = []
for mr in mean_results:
    ranks.append(rankdata(mr).tolist())
ranks = np.array(ranks)
print("\nRanks:\n", ranks)


Ranks:
 [[1.  5.  6.  2.  4.  3. ]
 [3.  5.  4.  2.  6.  1. ]
 [5.  6.  1.  3.  2.  4. ]
 [3.  2.  5.  4.  6.  1. ]
 [4.  3.  1.5 6.  1.5 5. ]
 [1.  2.  6.  3.  5.  4. ]
 [2.  3.  5.  4.  6.  1. ]
 [2.  3.  5.  4.  6.  1. ]
 [5.  2.  3.5 6.  3.5 1. ]
 [2.  3.  6.  4.  5.  1. ]
 [4.  6.  1.5 3.  1.5 5. ]
 [2.  6.  5.  1.  3.  4. ]]


In [6]:
mean_ranks = np.mean(ranks, axis=0)
mean_ranks

array([2.83333333, 3.83333333, 4.125     , 3.5       , 4.125     ,
       2.58333333])

## Wilcoxon's rank statistical test

In [7]:
from scipy.stats import ranksums

alfa = .05
w_statistic = np.zeros((len(clfs), len(clfs)))
p_value = np.zeros((len(clfs), len(clfs)))

for i in range(len(clfs)):
    for j in range(len(clfs)):
        w_statistic[i, j], p_value[i, j] = ranksums(ranks.T[i], ranks.T[j])

In [8]:
from tabulate import tabulate

headers = list(clfs.keys())
names_column = np.expand_dims(np.array(list(clfs.keys())), axis=1)
w_statistic_table = np.concatenate((names_column, w_statistic), axis=1)
w_statistic_table = tabulate(w_statistic_table, headers, floatfmt=".2f")
p_value_table = np.concatenate((names_column, p_value), axis=1)
p_value_table = tabulate(p_value_table, headers, floatfmt=".2f")
print("\nw-statistic:\n", w_statistic_table, "\n\np-value:\n", p_value_table)


w-statistic:
          CART    EBM    LR_l2    GNB     LR    DL
-----  ------  -----  -------  -----  -----  ----
CART     0.00  -1.50    -1.73  -1.04  -1.73  0.58
EBM      1.50   0.00    -0.32   0.29  -0.38  1.73
LR_l2    1.73   0.32     0.00   1.01  -0.12  2.22
GNB      1.04  -0.29    -1.01   0.00  -0.84  1.18
LR       1.73   0.38     0.12   0.84   0.00  2.19
DL      -0.58  -1.73    -2.22  -1.18  -2.19  0.00 

p-value:
          CART    EBM    LR_l2    GNB    LR    DL
-----  ------  -----  -------  -----  ----  ----
CART     1.00   0.13     0.08   0.30  0.08  0.56
EBM      0.13   1.00     0.75   0.77  0.71  0.08
LR_l2    0.08   0.75     1.00   0.31  0.91  0.03
GNB      0.30   0.77     0.31   1.00  0.40  0.24
LR       0.08   0.71     0.91   0.40  1.00  0.03
DL       0.56   0.08     0.03   0.24  0.03  1.00


In [9]:
advantage = np.zeros((len(clfs), len(clfs)))
advantage[w_statistic > 0] = 1
advantage_table = tabulate(np.concatenate(
    (names_column, advantage), axis=1), headers)
print("\nAdvantage:\n", advantage_table)


Advantage:
          CART    EBM    LR_l2    GNB    LR    DL
-----  ------  -----  -------  -----  ----  ----
CART        0      0        0      0     0     1
EBM         1      0        0      1     0     1
LR_l2       1      1        0      1     0     1
GNB         1      0        0      0     0     1
LR          1      1        1      1     0     1
DL          0      0        0      0     0     0


In [10]:
significance = np.zeros((len(clfs), len(clfs)))
significance[p_value <= alfa] = 1
significance_table = tabulate(np.concatenate(
    (names_column, significance), axis=1), headers)
print("\nStatistical significance (alpha = 0.05):\n", significance_table)


Statistical significance (alpha = 0.05):
          CART    EBM    LR_l2    GNB    LR    DL
-----  ------  -----  -------  -----  ----  ----
CART        0      0        0      0     0     0
EBM         0      0        0      0     0     0
LR_l2       0      0        0      0     0     1
GNB         0      0        0      0     0     0
LR          0      0        0      0     0     1
DL          0      0        1      0     1     0


In [12]:
stat_better = significance * advantage
stat_better_table = tabulate(np.concatenate(
    (names_column, stat_better), axis=1), headers)
print("Statistically significantly better:\n", stat_better_table)

Statistically significantly better:
          CART    EBM    LR_l2    GNB    LR    DL
-----  ------  -----  -------  -----  ----  ----
CART        0      0        0      0     0     0
EBM         0      0        0      0     0     0
LR_l2       0      0        0      0     0     1
GNB         0      0        0      0     0     0
LR          0      0        0      0     0     1
DL          0      0        0      0     0     0
